[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sagar87/MeyerBender/spatialproteomics_workshop/blob/main/notebooks/workshop02_task.ipynb)

# Downstream analysis of spatial proteomics data

In the previous exercise, you have seen which steps are required to transform high-dimensional image data into something more workable, such as a list of cells with associated cell types.
This is the prerequisite for any meaningful downstream analysis, the goal of which it is to find spatial patterns associated with a readout of interest, such as patient survival.
Here, we explore how one can go from individual cell types to constructing neighborhood profiles.

In [ ]:
# download the data
# if you have already run this cell once, there is no need to run it again

# use this when running on colab
! wget https://www.huber.embl.de/users/matthias/spatialproteomics_workshop_data.tar.gz /content/spatialproteomics_workshop_data.tar.gz
! tar -xzf /content/spatialproteomics_workshop_data.tar.gz
! pip install spatialproteomics==0.5.6
! pip install squidpy==1.2.2
data_dir = '/content/data'

# use this when running locally
#! wget https://www.huber.embl.de/users/matthias/spatialproteomics_workshop_data.tar.gz spatialproteomics_workshop_data.tar.gz
#! tar -xzf spatialproteomics_workshop_data.tar.gz
#data_dir = 'data'

In [ ]:
%reload_ext autoreload
%autoreload 2

import xarray as xr
import spatialproteomics as sp
import matplotlib.pyplot as plt
import scanpy as sc
from tqdm.auto import tqdm
from glob import glob
import seaborn as sns
import os

## Reading in files
Let's look at some lymph nodes. Segmentation and cell type prediction was already performed on this data, so we can simply read in the zarr files.

In [ ]:
channels = ['PAX5', 'CD3', 'CD11b', 'CD11c', 'CD15', 'CD68', 'Podoplanin', 'CD31', 'CD34', 'CD90', 'CD56']
quantiles = [0.8, 0.5, 0.8, 0.8, 0.8, 0.8, 0.95, 0.95, 0.95, 0.95, 0.8]
colors = ['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#ffd8b1', '#f58231', '#911eb4', '#fffac8', '#469990', '#fabed4', '#9A6324']
ct_marker_dict = {'B': 'PAX5', 'T': 'CD3', 'Myeloid': 'CD11b', 'Dendritic': 'CD11c', 'Granulo': 'CD15', 'Macro': 'CD68', 'Stroma PDPN': 'Podoplanin', 'Stroma CD31': 'CD31', 'Stroma CD34': 'CD34', 'Stroma CD90': 'CD90', 'NK': 'CD56'}

In [ ]:
# reading all of the images into a dictionary

sd_dict = {}
for sample_path in glob(os.path.join(data_dir, 'zarrs/*.zarr')):
    sample_id = sample_path.split('/')[-1].replace(".zarr", "")
    sd_dict[sample_id] = xr.open_dataset(sample_path, engine='zarr')

In [ ]:
# TODO: go through each sample and plot the intensities and the cell type predictions next to one another

## Exploration of marker profiles
Before we do any neighborhood analysis, let's verify that our cell type annotations are somewhat sensible. We can for example do this by plotting the expression matrix or UMAP of every sample. There are many ways to do this, however `scanpy` has become one of the most useful frameworks for this purpose. `Spatialproteomics` provides export functions to interact with other packages. Let's convert our `spatialproteomics` objects into `anndata` objects and use `scanpy` for a preliminary analysis.

In [ ]:
adata_dict = {}
for sample_id, sd_obj in sd_dict.items():
    # converting to anndata and performing umap computation
    adata = sd_obj.tl.convert_to_anndata()
    sc.pp.neighbors(adata)  # Compute neighbors
    sc.tl.umap(adata)       # Compute UMAP
    adata_dict[sample_id] = adata

In [ ]:
# TODO: look at the scanpy documentation and use their heatmap and umap functions to visualize the marker profiles

## Defining neighborhoods
Next up, we can define cellular neighborhoods. This boils down to two steps. 

In the first step, we count all cells within a specified radius of a cell. For example, if we look at cell A, we count all cells in a radius of 50 microns, and note down their relative frequencies. So instead of saying that a cell is a B cell, we can now say that it is in a neighborhood with 80% B cells and 20% T cells.

Once we have those neighborhood profiles, we can cluster them across all samples. There are many ways to do this, but in order to keep things simple, we only look at k-mean clustering here.

In [ ]:
from scipy.spatial import KDTree
import pandas as pd

# radius is in pixels here, each pixel is 0.5 microns across
def construct_neighborhood_df(df, x='centroid-1', y='centroid-0', label_col='_labels', radius=100):
    # Build KDTree for efficient neighborhood queries
    tree = KDTree(df[[x, y]])

    # Initialize a DataFrame to store the neighborhood counts
    cell_types = df[label_col].unique()
    neighborhood_profile = pd.DataFrame(0, index=range(len(df)), columns=cell_types)
    
    # resetting the index to start from 0, makes accessing with loc easier
    original_index = df.index
    df = df.reset_index()

    # Iterate over each cell
    for i in range(len(df)):
        # Query the KDTree for neighbors within the radius
        indices = tree.query_ball_point(df.loc[i, [x, y]], r=radius)

        # Remove the cell itself from the neighbors list
        indices = [idx for idx in indices if idx != i]

        # Count the cell types of the neighbors
        neighbor_counts = df.loc[indices, label_col].value_counts()

        # Update the neighborhood profile for the current cell
        for cell_type, count in neighbor_counts.items():
            neighborhood_profile.at[i, cell_type] = count

    # Normalize the neighborhood profile so that each row sums to 1
    neighborhood_profile = neighborhood_profile.div(neighborhood_profile.sum(axis=1), axis=0)
    
    # Add the centroids back to the neighborhood profile
    neighborhood_profile[x] = df[x]
    neighborhood_profile[y] = df[y]
    
    # setting the index back to the original ones
    neighborhood_profile.index = original_index

    return neighborhood_profile

In [ ]:
neighborhood_dfs = []

for sample_id, ds in tqdm(sd_dict.items()):
    spatial_df = ds.pp.get_layer_as_df()
    neighborhood_df = construct_neighborhood_df(spatial_df, radius=100)
    # adding the sample and cell index to the df
    neighborhood_df['cell'] = neighborhood_df.index
    neighborhood_df['sample'] = sample_id
    neighborhood_dfs.append(neighborhood_df)

big_neighborhood_df = pd.concat(neighborhood_dfs, ignore_index=True)

# it can happen that a cell type was not present in a sample
# in this case, the value will be NaN
# we replace those with 0
big_neighborhood_df = big_neighborhood_df.fillna(0)

In [ ]:
big_neighborhood_df

In [ ]:
# Select columns for clustering (exclude 'cell' and 'sample')
features = big_neighborhood_df.drop(columns=['cell', 'sample', 'centroid-0', 'centroid-1'])
features.head()

In [ ]:
from sklearn.cluster import KMeans

# TODO: run the k-means algorithm on the 'features' data frame. Put the cluster labels into the big_neighborhood_df
# Experiment with different k's.

In [ ]:
# TODO: compute the mean expression of each cluster and plot it as a heatmap (e. g. using seaborn)

In [ ]:
# TODO: at this point, big_neighborhood_df contains sample_id, centroid-1, centroid-0, and your neighborhood labels
# plot the centroids as a scatterplot (e. g. using seaborn) and color them according to the neighborhood. What do you observe?

In [ ]:
# TODO: how meaningful are our clusters? Try computing a PCA on the features and color the points according to the neighborhood.

## Additional analysis with squidpy
There are plenty of tools to analyse spatial data these days. Let's briefly look at how to use squidpy to create a neighborhood enrichment.

In [ ]:
# this is only on a single sample, you could also concatenate the adata objects to get a more global view
import squidpy as sq

adata = adata_dict['166_1_H3_LK'].copy()

In [ ]:
# formatting the anndata object. This is required for squidpy to work properly
adata.obsm['spatial'] = np.array(adata.obs[['centroid-0', 'centroid-1']])

In [ ]:
# TODO: use squidpy to perform a neighborhood enrichment. Look into other possible downstream methods offered by squidpy.